In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.style.use('science')

In [51]:
df = pd.read_csv("../data/results-benchmark.csv")

In [4]:
df.head()

,n,idx,distribution,travel,serv,cost_profile,dotsp,enum,lns,mtsp,...,svf,tsp,dotsp_gap,tsp_gap,svf_gap,mtsp_gap,lns_gap,nnsvf_gap,enum_gap,best
0,6,0,0,0,0,"(0.5, 2.5, 10)",416.0,395.078292,395.078,447.0,...,473.0,416.0,5.295663,5.295663,19.723194,13.142215,0.000000,7.067465,0.019821,395.078000
1,6,0,0,0,0,"(1.0, 2.5, 10)",481.0,460.767292,460.767,512.0,...,573.0,481.0,4.391156,4.391156,24.357864,11.119069,0.000000,8.731745,0.166803,460.767000
2,6,0,0,0,0,"(2.0, 2.5, 10)",610.0,592.145292,592.145,644.0,...,773.0,610.0,3.015309,3.015309,30.542350,8.757146,0.000000,11.121431,0.024543,592.145000
3,6,0,0,0,1,"(0.5, 2.5, 10)",683.0,621.380942,621.381,734.0,...,694.0,683.0,9.916471,9.916471,11.686721,18.123996,0.000009,10.560198,0.061343,621.380942
4,6,0,0,0,1,"(1.0, 2.5, 10)",748.0,691.553479,692.968,805.0,...,794.0,748.0,8.162279,8.162279,14.813969,16.404591,0.204543,12.934144,0.000000,691.553479


## Preprocess data

In [56]:
# Get the weight coefficient for the travel times
df['$\omega^T$'] = df.cost_profile.str[1:4]
df['$n$'] = df['n']

# Create new gap columns with algorithm names in uppercase 
for col in [col for col in df.columns if col.endswith("_gap")]:
    alg = col.removesuffix("_gap").upper() # algorithm name in uppercase
    df[alg] = df[col]
    
ALGS = ["LNS", "MTSP", "DOTSP", "NNSVF"]

## Small instances $(n \leq 10)$

In [196]:
res = df[df.n <= 10]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$', 'serv'])[ALGS].mean().round(2)
res = res.unstack().swaplevel(0, 1, axis=1).sort_index(axis=1)

# Fix the column ordering
order=[(0, 'LNS'), (0, 'MTSP'), (0, 'DOTSP'), (0, 'NNSVF'), (1,   'LNS'), (1,  'MTSP'), (1, 'DOTSP'), (1, 'NNSVF')]
res = res[order] 

res

serv               0                        1                    
                 LNS  MTSP DOTSP  NNSVF   LNS   MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                   
6   0.5         0.80  5.70  4.50  11.16  2.59  10.96  6.83  11.00
    1.0         0.73  4.82  3.43  13.16  1.96   8.15  5.65  12.70
    2.0         0.79  2.71  2.45  16.50  1.97   7.23  4.20  15.75
8   0.5         1.12  5.25  3.73   9.46  3.93  10.35  7.28   9.69
    1.0         0.84  4.47  2.88  12.17  2.84   7.90  5.68  11.39
    2.0         0.56  3.99  1.96  16.43  2.24   6.36  3.96  14.96
10  0.5         2.04  7.72  5.50  10.27  3.32  10.47  8.64  11.16
    1.0         1.14  5.86  4.10  13.71  3.00   8.55  6.82  13.64
    2.0         0.88  4.63  2.80  19.74  1.70   5.82  4.62  18.56

In [197]:
tmp = res.copy()

# Add dummy columns
tmp.insert(0, 'dum1', '')
tmp.insert(5, 'dum2', '')

print(tmp.to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{lllrrrrlrrrr}
\toprule
 & serv & dum1 & \multicolumn{4}{r}{0} & dum2 & \multicolumn{4}{r}{1} \\
 &  &  & LNS & MTSP & DOTSP & NNSVF &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{6} & 0.5 &  & 0.80 & 5.70 & 4.50 & 11.16 &  & 2.59 & 10.96 & 6.83 & 11.00 \\
 & 1.0 &  & 0.73 & 4.82 & 3.43 & 13.16 &  & 1.96 & 8.15 & 5.65 & 12.70 \\
 & 2.0 &  & 0.79 & 2.71 & 2.45 & 16.50 &  & 1.97 & 7.23 & 4.20 & 15.75 \\
\cline{1-12}
\multirow[t]{3}{*}{8} & 0.5 &  & 1.12 & 5.25 & 3.73 & 9.46 &  & 3.93 & 10.35 & 7.28 & 9.69 \\
 & 1.0 &  & 0.84 & 4.47 & 2.88 & 12.17 &  & 2.84 & 7.90 & 5.68 & 11.39 \\
 & 2.0 &  & 0.56 & 3.99 & 1.96 & 16.43 &  & 2.24 & 6.36 & 3.96 & 14.96 \\
\cline{1-12}
\multirow[t]{3}{*}{10} & 0.5 &  & 2.04 & 7.72 & 5.50 & 10.27 &  & 3.32 & 10.47 & 8.64 & 11.16 \\
 & 1.0 &  & 1.14 & 5.86 & 4.10 & 13.71 &  & 3.00 & 8.55 & 6.82 & 13.64 \\
 & 2.0 &  & 0.88 & 4.63 & 2.80 & 19.74 &  & 1.70 & 5.82 & 4.62 & 18.56 \\
\

serv               0                        1                    
                 LNS  MTSP DOTSP  NNSVF   LNS   MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                   
6   0.5         0.80  5.70  4.50  11.16  2.59  10.96  6.83  11.00
    1.0         0.73  4.82  3.43  13.16  1.96   8.15  5.65  12.70
    2.0         0.79  2.71  2.45  16.50  1.97   7.23  4.20  15.75
8   0.5         1.12  5.25  3.73   9.46  3.93  10.35  7.28   9.69
    1.0         0.84  4.47  2.88  12.17  2.84   7.90  5.68  11.39
    2.0         0.56  3.99  1.96  16.43  2.24   6.36  3.96  14.96
10  0.5         2.04  7.72  5.50  10.27  3.32  10.47  8.64  11.16
    1.0         1.14  5.86  4.10  13.71  3.00   8.55  6.82  13.64
    2.0         0.88  4.63  2.80  19.74  1.70   5.82  4.62  18.56

In [200]:
res.mean().reset_index().groupby("level_1").mean()

,serv,0
level_1,,
DOTSP,0.5,4.723889
LNS,0.5,1.802778
MTSP,0.5,6.718889
NNSVF,0.5,13.413889


## Large instances $(n \ge 15)$

In [201]:
res = df[df.n >= 15]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$', 'serv'])[ALGS].mean().round(2)
res = res.unstack().swaplevel(0, 1, axis=1).sort_index(axis=1)

# Fix the column ordering
order=[(0, 'LNS'), (0, 'MTSP'), (0, 'DOTSP'), (0, 'NNSVF'), (1,   'LNS'), (1,  'MTSP'), (1, 'DOTSP'), (1, 'NNSVF')]
res = res[order] 

res

serv               0                        1                   
                 LNS  MTSP DOTSP  NNSVF   LNS  MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                  
15  0.5         0.16  3.75  2.85   8.95  0.16  5.76  4.10   6.41
    1.0         0.16  2.45  2.03  13.59  0.09  4.04  3.26  10.13
    2.0         0.70  1.88  1.70  22.15  0.18  2.79  2.19  17.01
20  0.5         0.28  3.41  2.29   8.53  0.03  5.99  5.20   7.21
    1.0         0.53  2.85  1.88  13.83  0.20  3.74  3.76  10.57
    2.0         1.44  1.69  1.36  22.91  0.21  2.76  2.78  18.15
25  0.5         0.30  2.13  1.67   7.98  0.00  5.96  5.41   7.26
    1.0         1.01  1.00  1.07  13.12  0.03  4.80  4.61  11.34
    2.0         1.34  1.22  0.58  22.48  0.10  3.33  2.98  18.43
30  0.5         0.21  3.11  1.64   7.11  0.00  5.82  5.40   5.98
    1.0         0.70  2.09  0.84  12.22  0.00  5.55  4.42   9.68
    2.0         3.49  1.05  0.27  22.01  0.01  3.67  2.47  16.15
35  0.5         0.61  1.81  0.86   6.64  0.01  6.19  5.26   6.57
    1.0         1.95  1.39  0.47  11.92  0.11  4.87  4.14  10.21
    2.0         4.97  1.25  0.17  21.68  0.34  3.65  2.85  17.66
40  0.5         0.57  1.80  0.46   5.99  0.00  7.19  6.10   6.68
    1.0         1.77  1.46  0.11  11.31  0.00  6.18  4.98  10.26
    2.0         5.85  1.04  0.06  21.48  0.00  4.15  2.81  16.65

In [202]:
tmp = res.copy()

# Add dummy columns
tmp.insert(0, 'dum1', '')
tmp.insert(5, 'dum2', '')

print(tmp.to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{lllrrrrlrrrr}
\toprule
 & serv & dum1 & \multicolumn{4}{r}{0} & dum2 & \multicolumn{4}{r}{1} \\
 &  &  & LNS & MTSP & DOTSP & NNSVF &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{15} & 0.5 &  & 0.16 & 3.75 & 2.85 & 8.95 &  & 0.16 & 5.76 & 4.10 & 6.41 \\
 & 1.0 &  & 0.16 & 2.45 & 2.03 & 13.59 &  & 0.09 & 4.04 & 3.26 & 10.13 \\
 & 2.0 &  & 0.70 & 1.88 & 1.70 & 22.15 &  & 0.18 & 2.79 & 2.19 & 17.01 \\
\cline{1-12}
\multirow[t]{3}{*}{20} & 0.5 &  & 0.28 & 3.41 & 2.29 & 8.53 &  & 0.03 & 5.99 & 5.20 & 7.21 \\
 & 1.0 &  & 0.53 & 2.85 & 1.88 & 13.83 &  & 0.20 & 3.74 & 3.76 & 10.57 \\
 & 2.0 &  & 1.44 & 1.69 & 1.36 & 22.91 &  & 0.21 & 2.76 & 2.78 & 18.15 \\
\cline{1-12}
\multirow[t]{3}{*}{25} & 0.5 &  & 0.30 & 2.13 & 1.67 & 7.98 &  & 0.00 & 5.96 & 5.41 & 7.26 \\
 & 1.0 &  & 1.01 & 1.00 & 1.07 & 13.12 &  & 0.03 & 4.80 & 4.61 & 11.34 \\
 & 2.0 &  & 1.34 & 1.22 & 0.58 & 22.48 &  & 0.10 & 3.33 & 2.98 & 18.43 \\
\cline

serv               0                        1                   
                 LNS  MTSP DOTSP  NNSVF   LNS  MTSP DOTSP  NNSVF
$n$ $\omega^T$                                                  
15  0.5         0.16  3.75  2.85   8.95  0.16  5.76  4.10   6.41
    1.0         0.16  2.45  2.03  13.59  0.09  4.04  3.26  10.13
    2.0         0.70  1.88  1.70  22.15  0.18  2.79  2.19  17.01
20  0.5         0.28  3.41  2.29   8.53  0.03  5.99  5.20   7.21
    1.0         0.53  2.85  1.88  13.83  0.20  3.74  3.76  10.57
    2.0         1.44  1.69  1.36  22.91  0.21  2.76  2.78  18.15
25  0.5         0.30  2.13  1.67   7.98  0.00  5.96  5.41   7.26
    1.0         1.01  1.00  1.07  13.12  0.03  4.80  4.61  11.34
    2.0         1.34  1.22  0.58  22.48  0.10  3.33  2.98  18.43
30  0.5         0.21  3.11  1.64   7.11  0.00  5.82  5.40   5.98
    1.0         0.70  2.09  0.84  12.22  0.00  5.55  4.42   9.68
    2.0         3.49  1.05  0.27  22.01  0.01  3.67  2.47  16.15
35  0.5         0.61  1.81  0.86   6.64  0.01  6.19  5.26   6.57
    1.0         1.95  1.39  0.47  11.92  0.11  4.87  4.14  10.21
    2.0         4.97  1.25  0.17  21.68  0.34  3.65  2.85  17.66
40  0.5         0.57  1.80  0.46   5.99  0.00  7.19  6.10   6.68
    1.0         1.77  1.46  0.11  11.31  0.00  6.18  4.98  10.26
    2.0         5.85  1.04  0.06  21.48  0.00  4.15  2.81  16.65

In [203]:
res.mean().reset_index().groupby("level_1").mean()

,serv,0
level_1,,
DOTSP,0.5,2.584167
LNS,0.5,0.764167
MTSP,0.5,3.383889
NNSVF,0.5,12.784722
